## The first big project - Professionally You!

### And, Tool use.

### But first: introducing Pushover

Pushover is a nifty tool for sending Push Notifications to your phone.

It's super easy to set up and install!

Simply visit https://pushover.net/ and click 'Login or Signup' on the top right to sign up for a free account, and create your API keys.

Once you've signed up, on the home screen, click "Create an Application/API Token", and give it any name (like Agents) and click Create Application.

Then add 2 lines to your `.env` file:

PUSHOVER_USER=_put the key that's on the top right of your Pushover home screen and probably starts with a u_  
PUSHOVER_TOKEN=_put the key when you click into your new application called Agents (or whatever) and probably starts with an a_

Remember to save your `.env` file, and run `load_dotenv(override=True)` after saving, to set your environment variables.

Finally, click "Add Phone, Tablet or Desktop" to install on your phone.

In [21]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr
import sqlite3
import re
import numpy as np
import time

# RAG dependencies - install if needed
try:
    import faiss
except ImportError:
    print("Installing faiss-cpu...")
    os.system("uv add faiss-cpu")
    import faiss

try:
    from rank_bm25 import BM25Okapi
except ImportError:
    print("Installing rank-bm25...")
    os.system("uv add rank-bm25")
    from rank_bm25 import BM25Okapi

In [22]:
# The usual start

load_dotenv(override=True)
openai = OpenAI()

In [23]:
# For pushover

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

Pushover user found and starts with u
Pushover token found and starts with a


In [24]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [25]:
push("HEY!!")

Push: HEY!!


In [26]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [27]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [28]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [29]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [30]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [31]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of this user'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['quest

In [32]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [33]:
globals()["record_unknown_question"]("this is a really hard question")

Push: Recording this is a really hard question asked that I couldn't answer


{'recorded': 'ok'}

In [34]:
# This is a more elegant way that avoids the IF statement.

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [35]:
reader = PdfReader("me/cameronbell2.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("me/summary2.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Cameron Bell"

# Initialize SQLite database
conn = sqlite3.connect('qa_database.db')
cursor = conn.cursor()

# Create Q&A table
cursor.execute('''
CREATE TABLE IF NOT EXISTS qa (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    question TEXT UNIQUE,
    answer TEXT,
    category TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
''')

# Seed with some sample Q&A based on your profile
sample_qa = [
    ("Where are you from?", "I'm from Southampton, Bermuda, but I've lived in England for 10 years and Madrid, Spain for a year during my MSc.", "background"),
    ("What's your educational background?", "I attended Ardingly College for boarding school, completed my International Baccalaureate, then went to University of Bristol for my MSc in Computer Science and Business Technology.", "education"),
    ("What are your hobbies?", "I love all foods, particularly trying new cuisine. I'm also a gym enthusiast and enjoy weights and calisthenics. I'm a big football fan too!", "personal"),
    ("Why did you move back to Bermuda?", "In 2022, I returned to Bermuda to be with my ageing grandfather and support my family.", "background"),
]

for q, a, cat in sample_qa:
    cursor.execute("INSERT OR IGNORE INTO qa (question, answer, category) VALUES (?, ?, ?)", (q, a, cat))

conn.commit()
print("Database initialized with sample Q&A")

def search_qa(user_query, limit=3):
    """Search for relevant Q&A based on keyword matching"""
    keywords = re.findall(r'\b\w+\b', user_query.lower())
    keyword_pattern = '%' + '%'.join(keywords[:3]) + '%'
    
    cursor.execute('''
        SELECT question, answer, category 
        FROM qa 
        WHERE question LIKE ? OR answer LIKE ?
        LIMIT ?
    ''', (keyword_pattern, keyword_pattern, limit))
    
    results = cursor.fetchall()
    
    if results:
        context = "Relevant Q&A from knowledge base:\n"
        for q, a, cat in results:
            context += f"Q: {q}\nA: {a}\n\n"
        return context
    return None

Ignoring wrong pointing object 110 0 (offset 0)
Ignoring wrong pointing object 113 0 (offset 0)
Ignoring wrong pointing object 115 0 (offset 0)
Ignoring wrong pointing object 117 0 (offset 0)
Ignoring wrong pointing object 119 0 (offset 0)
Ignoring wrong pointing object 123 0 (offset 0)
Ignoring wrong pointing object 124 0 (offset 0)


Database initialized with sample Q&A


## RAG Solutions - 3 Approaches

Below are 3 different RAG implementations for retrieving relevant information from my documents.


In [36]:
# Shared function: Chunk documents

def chunk_documents(chunk_size=500, overlap=50):
    """Chunk all documents into smaller pieces for retrieval"""
    # Read CV (already have linkedin from cameronbell2.pdf loaded above)
    cv_reader = PdfReader("me/cameronbell_cv.pdf")
    cv = ""
    for page in cv_reader.pages:
        text = page.extract_text()
        if text:
            cv += text
    
    # Combine all text
    all_text = summary + "\n\n" + linkedin + "\n\n" + cv
    
    # Split into chunks
    chunks = []
    start = 0
    while start < len(all_text):
        end = start + chunk_size
        chunk = all_text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap
    
    return chunks

# Test chunking
doc_chunks = chunk_documents()
print(f"Created {len(doc_chunks)} chunks from all documents")
print(f"First chunk preview: {doc_chunks[0][:100]}...")


Created 71 chunks from all documents
First chunk preview: My name is Cameron Bell. I'm an AI / ML engineer, software developer and data scientist. I'm origina...


### Solution 1: OpenAI Embeddings + FAISS Vector Store


In [37]:
# Solution 1: OpenAI Embeddings + FAISS

# Build vector store
print("Building FAISS vector store with OpenAI embeddings...")
chunks = chunk_documents()

# Generate embeddings
embeddings = []
for chunk in chunks:
    response = openai.embeddings.create(
        model="text-embedding-3-small",
        input=chunk
    )
    embeddings.append(response.data[0].embedding)

# Convert to numpy array
embeddings_array = np.array(embeddings).astype('float32')
dimension = embeddings_array.shape[1]

# Create FAISS index
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(embeddings_array)

print(f"Created FAISS index with {faiss_index.ntotal} vectors")

def search_embeddings(query, top_k=3):
    """Search using semantic embeddings"""
    # Embed the query
    response = openai.embeddings.create(
        model="text-embedding-3-small",
        input=query
    )
    query_embedding = np.array([response.data[0].embedding]).astype('float32')
    
    # Search FAISS index
    distances, indices = faiss_index.search(query_embedding, top_k)
    
    # Get top chunks
    results = [chunks[idx] for idx in indices[0]]
    
    context = f"Relevant chunks from knowledge base (Embeddings):\n\n"
    for i, chunk in enumerate(results, 1):
        context += f"{i}. {chunk[:200]}...\n\n"
    
    return context

# Test
test_query = "What are your technical skills?"
print(search_embeddings(test_query, top_k=2))


Building FAISS vector store with OpenAI embeddings...
Created FAISS index with 71 vectors
Relevant chunks from knowledge base (Embeddings):

1. y queries and computer 
skills. I have always had an affection for elderly citizens due to my close 
relationship with my grandparents. This was a way to help teach them key 
technological skills, to ...

2. pt, with hands-on experience in RESTful APIs, full-stack 
applications, and cloud deployment AWS, GCP, Azure). Proficient with FastAPI, 
Flask, React, and CI/CD pipelines GitHub Actions, Docker). St...




In [50]:
# Updated comparison function using the fixed FTS5
def compare_rag_solutions_fixed(test_query):
    """Compare all 3 RAG solutions on the same query - using fixed FTS5"""
    print(f"=" * 80)
    print(f"Query: {test_query}")
    print(f"=" * 80)
    print()
    
    # Solution 1: Embeddings
    print("📊 Solution 1: OpenAI Embeddings + FAISS")
    print("-" * 80)
    start = time.time()
    result1 = search_embeddings(test_query, top_k=2)
    time1 = time.time() - start
    print(result1)
    print(f"⏱️  Time: {time1:.3f}s")
    print()
    
    # Solution 2: BM25
    print("📊 Solution 2: BM25 Text Search")
    print("-" * 80)
    start = time.time()
    result2 = search_bm25(test_query, top_k=2)
    time2 = time.time() - start
    print(result2)
    print(f"⏱️  Time: {time2:.3f}s")
    print()
    
    # Solution 3: FTS5 (Fixed)
    print("📊 Solution 3: SQLite FTS5 (Fixed)")
    print("-" * 80)
    start = time.time()
    result3 = search_fts5_fixed(test_query, top_k=2)
    time3 = time.time() - start
    print(result3)
    print(f"⏱️  Time: {time3:.3f}s")
    print()
    
    # Summary
    print("=" * 80)
    print("Summary:")
    print(f"Embeddings: {time1:.3f}s | BM25: {time2:.3f}s | FTS5: {time3:.3f}s")
    print("=" * 80)

# Test with the fixed function
print("\n🔄 Test 1: Education Question (Fixed)")
compare_rag_solutions_fixed("Where did you study and what degree do you have")



🔄 Test 1: Education Question (Fixed)
Query: Where did you study and what degree do you have

📊 Solution 1: OpenAI Embeddings + FAISS
--------------------------------------------------------------------------------
Relevant chunks from knowledge base (Embeddings):

1. perations under time constraints
Led public speaking and engagement efforts (guided tours, group
presentations, marketing outreach), enhancing confidence and the ability to
synthesize technical and no...

2.  & Artificial Intelligence, Cloud Foundations 
UNIVERSITY OF BRISTOL                  BRISTOL, UNITED KINGDOM 
Bachelor of Science, Management                            SEP 2017 – JUL 2020 
• Modules...


⏱️  Time: 0.393s

📊 Solution 2: BM25 Text Search
--------------------------------------------------------------------------------
Relevant chunks from knowledge base (BM25):

1. deling or business recommendation and require proficiency 
in Python/ML systems.
AI / ML Engineer CV Bullet P oint s
1Final Recommendation:

### Solution 2: BM25 Text Search


In [51]:
# Solution 2: BM25 Text Search

print("Building BM25 index...")
chunks = chunk_documents()

# Tokenize chunks (simple split on whitespace)
tokenized_chunks = [chunk.lower().split() for chunk in chunks]

# Build BM25 index
bm25_index = BM25Okapi(tokenized_chunks)
print(f"Created BM25 index with {len(tokenized_chunks)} chunks")

def search_bm25(query, top_k=3):
    """Search using BM25 ranking"""
    # Tokenize query
    query_tokens = query.lower().split()
    
    # Get BM25 scores
    scores = bm25_index.get_scores(query_tokens)
    
    # Get top-k indices
    top_indices = np.argsort(scores)[::-1][:top_k]
    
    # Get top chunks
    results = [chunks[idx] for idx in top_indices]
    
    context = f"Relevant chunks from knowledge base (BM25):\n\n"
    for i, chunk in enumerate(results, 1):
        context += f"{i}. {chunk[:200]}...\n\n"
    
    return context

# Test
test_query = "What are your technical skills?"
print(search_bm25(test_query, top_k=2))


Building BM25 index...
Created BM25 index with 71 chunks
Relevant chunks from knowledge base (BM25):

1. entist 
roles. If you target Data Science, focus on roles with a strong "ML Systems" 
or "Advanced Modeling" component.
Do not target junior roles exclusively. Apply for AI/ML Engineer positions 
aski...

2. rrent CV strength is likely at the strong Junior/Entry-Level ML/AI 
Engineer level, but with the ability to compete for roles that require 13 
years of experience, especially if they are looking for ...




### Solution 3: SQLite FTS5 Full-Text Search


In [60]:
# Solution 3: SQLite FTS5 Full-Text Search

# Create FTS5 table
cursor.execute('''
    CREATE VIRTUAL TABLE IF NOT EXISTS knowledge_base USING fts5(
        content,
        source,
        chunk_id
    )
''')

# Populate FTS5 table
chunks = chunk_documents()
print("Populating FTS5 table...")

for i, chunk in enumerate(chunks):
    source = "combined_docs"
    cursor.execute(
        "INSERT INTO knowledge_base (content, source, chunk_id) VALUES (?, ?, ?)",
        (chunk, source, i)
    )

conn.commit()
print(f"Inserted {len(chunks)} chunks into FTS5 table")

def search_fts5(query, top_k=3):
    """Search using SQLite FTS5"""
    # 1) sanitize query for FTS5 parser (remove characters that cause MATCH syntax errors)
    cleaned = re.sub(r'["\'`~*^()]+', ' ', query).replace('?', ' ').lower()
    terms = re.findall(r"\w+", cleaned)
    if not terms:
        return None
    fts_expr = " OR ".join(terms)  # e.g., technical OR skills


    # 2) safely quote for SQL, then inline LIMIT
    q = conn.execute("SELECT quote(?)", (cleaned,)).fetchone()[0]
    top_k_int = max(1, int(top_k))
    sql = f"""
        SELECT content, bm25(knowledge_base) as rank
        FROM knowledge_base
        WHERE knowledge_base MATCH {fts_expr}
        ORDER BY bm25(knowledge_base)
        LIMIT {top_k_int}
    """
    cursor.execute(sql)
    results = cursor.fetchall()
    
    if results:
        context = f"Relevant chunks from knowledge base (FTS5):\n\n"
        for i, (chunk, rank) in enumerate(results, 1):
            context += f"{i}. [Rank: {rank:.2f}] {chunk[:200]}...\n\n"
        return context
    return None

# Test
test_query = "technical skills"
print(search_fts5(test_query, top_k=2))


Populating FTS5 table...
Inserted 71 chunks into FTS5 table


OperationalError: no such column: skills

### Comparison: All 3 Solutions Side-by-Side


In [58]:
def compare_rag_solutions(test_query):
    """Compare all 3 RAG solutions on the same query"""
    print(f"=" * 80)
    print(f"Query: {test_query}")
    print(f"=" * 80)
    print()
    
    # Solution 1: Embeddings
    print("📊 Solution 1: OpenAI Embeddings + FAISS")
    print("-" * 80)
    start = time.time()
    result1 = search_embeddings(test_query, top_k=2)
    time1 = time.time() - start
    print(result1)
    print(f"⏱️  Time: {time1:.3f}s")
    print()
    
    # Solution 2: BM25
    print("📊 Solution 2: BM25 Text Search")
    print("-" * 80)
    start = time.time()
    result2 = search_bm25(test_query, top_k=2)
    time2 = time.time() - start
    print(result2)
    print(f"⏱️  Time: {time2:.3f}s")
    print()
    
    # Solution 3: FTS5
    print("📊 Solution 3: SQLite FTS5")
    print("-" * 80)
    start = time.time()
    result3 = search_fts5(test_query, top_k=2)
    time3 = time.time() - start
    print(result3)
    print(f"⏱️  Time: {time3:.3f}s")
    print()
    
    # Summary
    print("=" * 80)
    print("Summary:")
    print(f"Embeddings: {time1:.3f}s | BM25: {time2:.3f}s | FTS5: {time3:.3f}s")
    print("=" * 80)

# Test with different queries
print("\n🔄 Test 1: Education Question")
compare_rag_solutions("Where did you study and what degree do you have?")

print("\n\n🔄 Test 2: Technical Skills")
compare_rag_solutions("What are your technical skills and programming languages?")

print("\n\n🔄 Test 3: Personal Background")
compare_rag_solutions("Tell me about your background and where you're from")



🔄 Test 1: Education Question
Query: Where did you study and what degree do you have?

📊 Solution 1: OpenAI Embeddings + FAISS
--------------------------------------------------------------------------------
Relevant chunks from knowledge base (Embeddings):

1. perations under time constraints
Led public speaking and engagement efforts (guided tours, group
presentations, marketing outreach), enhancing confidence and the ability to
synthesize technical and no...

2. 17 - 2020)
  Page 4 of 5   
Ardingly College
International Baccalaureate  · (2015 - 2017)
Warwick Academy
I/GCSE's - 4 A*s, 3 A's, 1 B, 1 C  · (2013 - 2015)
  Page 5 of 505_Master_CV_Bank.md
Cameron B...


⏱️  Time: 0.262s

📊 Solution 2: BM25 Text Search
--------------------------------------------------------------------------------
Relevant chunks from knowledge base (BM25):

1. entist 
roles. If you target Data Science, focus on roles with a strong "ML Systems" 
or "Advanced Modeling" component.
Do not target junior roles

OperationalError: fts5: syntax error near "?"

## RAG Solutions Summary

You now have 3 working RAG implementations:

1. **Solution 1 (Embeddings)**: Best semantic understanding - finds conceptually similar content even without exact keyword matches. Requires OpenAI API for embeddings (~$0.0001/1K tokens).

2. **Solution 2 (BM25)**: Fast keyword-based search. Works offline, no API costs. Great for exact keyword matching.

3. **Solution 3 (FTS5)**: SQL-based full-text search. Built into SQLite, persistent storage, good for structured queries.

**To use in chat**: Replace `search_qa(message)` in the chat function with any of these:
- `search_embeddings(message)` - for semantic search
- `search_bm25(message)` - for fast keyword search  
- `search_fts5(message)` - for SQL-based search

Or combine them for hybrid retrieval!


In [ ]:
def get_system_prompt(qa_context=""):
    system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

    system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
    
    if qa_context:
        system_prompt += f"\n{qa_context}\n"
    
    system_prompt += f"With this context, please chat with the user, always staying in character as {name}."
    return system_prompt

system_prompt = get_system_prompt()


In [ ]:
def chat(message, history):
    # Search Q&A database for relevant context
    qa_context = search_qa(message)
    
    # Build system prompt with Q&A context
    prompt = get_system_prompt(qa_context)
    
    messages = [{"role": "system", "content": prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Tool called: record_unknown_question
Push: Recording do you have a patent? asked that I couldn't answer
Tool called: record_unknown_question
Push: Recording Do you have a patent? asked that I couldn't answer
Tool called: record_user_details
Push: Recording interest from Name not provided with email cameronsobell@gmail.com and notes not provided


## And now for deployment

This code is in `app.py`

We will deploy to HuggingFace Spaces.

Before you start: remember to update the files in the "me" directory - your LinkedIn profile and summary.txt - so that it talks about you! Also change `self.name = "Ed Donner"` in `app.py`..  

Also check that there's no README file within the 1_foundations directory. If there is one, please delete it. The deploy process creates a new README file in this directory for you.

1. Visit https://huggingface.co and set up an account  
2. From the Avatar menu on the top right, choose Access Tokens. Choose "Create New Token". Give it WRITE permissions - it needs to have WRITE permissions! Keep a record of your new key.  
3. In the Terminal, run: `uv tool install 'huggingface_hub[cli]'` to install the HuggingFace tool, then `hf auth login` to login at the command line with your key. Afterwards, run `hf auth whoami` to check you're logged in  
4. Take your new token and add it to your .env file: `HF_TOKEN=hf_xxx` for the future
5. From the 1_foundations folder, enter: `uv run gradio deploy` 
6. Follow its instructions: name it "career_conversation", specify app.py, choose cpu-basic as the hardware, say Yes to needing to supply secrets, provide your openai api key, your pushover user and token, and say "no" to github actions.  

Thank you Robert, James, Martins, Andras and Priya for these tips.  
Please read the next 2 sections - how to change your Secrets, and how to redeploy your Space (you may need to delete the README.md that gets created in this 1_foundations directory).

#### More about these secrets:

If you're confused by what's going on with these secrets: it just wants you to enter the key name and value for each of your secrets -- so you would enter:  
`OPENAI_API_KEY`  
Followed by:  
`sk-proj-...`  

And if you don't want to set secrets this way, or something goes wrong with it, it's no problem - you can change your secrets later:  
1. Log in to HuggingFace website  
2. Go to your profile screen via the Avatar menu on the top right  
3. Select the Space you deployed  
4. Click on the Settings wheel on the top right  
5. You can scroll down to change your secrets (Variables and Secrets section), delete the space, etc.

#### And now you should be deployed!

If you want to completely replace everything and start again with your keys, you may need to delete the README.md that got created in this 1_foundations folder.

Here is mine: https://huggingface.co/spaces/ed-donner/Career_Conversation

I just got a push notification that a student asked me how they can become President of their country 😂😂

For more information on deployment:

https://www.gradio.app/guides/sharing-your-app#hosting-on-hf-spaces

To delete your Space in the future:  
1. Log in to HuggingFace
2. From the Avatar menu, select your profile
3. Click on the Space itself and select the settings wheel on the top right
4. Scroll to the Delete section at the bottom
5. ALSO: delete the README file that Gradio may have created inside this 1_foundations folder (otherwise it won't ask you the questions the next time you do a gradio deploy)


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• First and foremost, deploy this for yourself! It's a real, valuable tool - the future resume..<br/>
            • Next, improve the resources - add better context about yourself. If you know RAG, then add a knowledge base about you.<br/>
            • Add in more tools! You could have a SQL database with common Q&A that the LLM could read and write from?<br/>
            • Bring in the Evaluator from the last lab, and add other Agentic patterns.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">Aside from the obvious (your career alter-ego) this has business applications in any situation where you need an AI assistant with domain expertise and an ability to interact with the real world.
            </span>
        </td>
    </tr>
</table>